In [179]:
import zeroberto_clustering
import datasets_handler
import zeroberto
import evaluation_metrics
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier

import pandas as pd

In [180]:
which_dataset = 'ag_news'
sample_size = 500

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)
raw_data[data_col] = raw_data[data_col].dropna()
classes_list = raw_data[class_col].drop_duplicates().to_list()
raw_data = evaluation_metrics.Encoder(raw_data,[class_col])

Found cached dataset ag_news (/Users/alealcoforado/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

In [181]:
train_dataset = raw_data.sample(sample_size,random_state=422)
X = train_dataset[data_col].to_list()
Y_class =  train_dataset[class_col].to_list()
Y_code =  train_dataset[class_col+"_code"].to_list()
pd.Series(zip(Y_class,Y_code)).drop_duplicates()

0    (science and technology, 1)
1                    (sports, 2)
2                  (business, 0)
3                     (world, 3)
dtype: object

In [182]:
classes_list_encoded = evaluation_metrics.Encoder(pd.DataFrame({"classes":classes_list}))['classes_code']

In [183]:
model = zeroberto.ZeroBERTo(classes_list=classes_list)#,hypothesis_template="{}.")##X## COMENTAR AQUI 

In [184]:
X_emb = model.encode(X)

In [185]:
classes_list_emb = model.encode(classes_list)


# KNN

In [186]:
# knn_regressor = KNeighborsRegressor(n_neighbors=len(classes_list), weights='uniform',
#                                      algorithm='auto', leaf_size=30, p=4, 
#                                      metric='minkowski', metric_params=None, n_jobs=None)

In [187]:
knn = KNeighborsClassifier(n_neighbors=len(classes_list), weights='distance',
                                     algorithm='ball_tree', leaf_size=30, p=4, 
                                     metric='minkowski', metric_params=None, n_jobs=None)

In [188]:
# list(zip(classes_list_emb,classes_list_encoded))

In [189]:
knn.fit(classes_list_emb,classes_list_encoded)


KNeighborsClassifier(algorithm='ball_tree', n_neighbors=4, p=4,
                     weights='distance')

In [190]:
print(knn.score(X_emb,Y_code))
print((knn.score(X_emb,Y_code))*len(train_dataset))

0.308
154.0


In [191]:
def getTopLabel(row):
  return row.idxmax()
def getTopProb(row):
  return row.max()

In [192]:
y_pred = knn.predict(X_emb)
y_probs = knn.predict_proba(X_emb)
y_probs

array([[0.21193863, 0.31159865, 0.25376964, 0.22269308],
       [0.23802856, 0.24737737, 0.27527199, 0.23932208],
       [0.22754104, 0.28478785, 0.25098733, 0.23668378],
       ...,
       [0.21460328, 0.30520341, 0.25868683, 0.22150647],
       [0.22590271, 0.27454012, 0.26049947, 0.2390577 ],
       [0.23295727, 0.26383629, 0.26138091, 0.24182552]])

In [193]:

df_probs = pd.DataFrame(y_probs,columns=classes_list)

label_results = df_probs.apply(getTopLabel,axis=1)
prob_results = df_probs.apply(getTopProb,axis=1)

In [194]:
label_results_df = pd.Series(label_results,name='prediction')
true_labels_df = raw_data[raw_data.index.isin(train_dataset.index)][class_col]
label_results_df.index=true_labels_df.index

final_result_df = pd.concat([true_labels_df,label_results_df],axis=1)
vector_true = ( final_result_df[class_col].astype(str).str.strip().str.lower() 
               == final_result_df['prediction'].astype(str).str.strip().str.lower() )
final_result_df_encoded = evaluation_metrics.Encoder(final_result_df,['prediction',class_col])
df_predictions_probs = pd.concat([final_result_df_encoded,
                                  pd.Series(prob_results,name='top_probability'),
                                  pd.Series(vector_true,name='right_wrong')],axis=1)

In [202]:
list(zip(label_results,Y_class))

[('science and technology', 'science and technology'),
 ('sports', 'sports'),
 ('science and technology', 'business'),
 ('science and technology', 'world'),
 ('science and technology', 'science and technology'),
 ('science and technology', 'sports'),
 ('science and technology', 'sports'),
 ('science and technology', 'business'),
 ('science and technology', 'business'),
 ('science and technology', 'world'),
 ('science and technology', 'business'),
 ('sports', 'world'),
 ('science and technology', 'world'),
 ('science and technology', 'sports'),
 ('science and technology', 'sports'),
 ('science and technology', 'business'),
 ('sports', 'sports'),
 ('science and technology', 'science and technology'),
 ('science and technology', 'business'),
 ('science and technology', 'science and technology'),
 ('science and technology', 'business'),
 ('science and technology', 'sports'),
 ('science and technology', 'sports'),
 ('science and technology', 'science and technology'),
 ('science and technol

In [205]:
all_metrics = evaluation_metrics.get_metrics(label_results.to_list(),Y_class)
all_metrics

/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'weighted': [{'accuracy': 0.308},
  {'precision': 0.483503921858813},
  {'recall': 0.308},
  {'f1': 0.1951882458933066}],
 'macro': [{'accuracy': 0.308},
  {'precision': 0.48353559271866214},
  {'recall': 0.295739079495929},
  {'f1': 0.1894601972105632}]}

In [206]:
# list(zip(label_results,Y_code,vector_true))[0:20]

In [207]:
df_predictions_probs.sort_values(['top_probability','prediction'], ascending=False)[['right_wrong']].sum(),len(df_predictions_probs)

(right_wrong    137
 dtype: object,
 999)

In [209]:
df_predictions_probs

/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/IPython/core/formatters.py:342: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,label_text,prediction,prediction_code,label_text_code,top_probability,right_wrong
445,science and technology,science and technology,1.0,1.0,0.315288,True
840,world,sports,2.0,3.0,NaN,False
1163,sports,science and technology,1.0,2.0,NaN,False
1192,sports,science and technology,1.0,2.0,NaN,False
1506,science and technology,science and technology,1.0,1.0,NaN,True
...,...,...,...,...,...,...
495,NaN,NaN,NaN,NaN,0.276803,NaN
496,NaN,NaN,NaN,NaN,0.292779,NaN
497,NaN,NaN,NaN,NaN,0.305203,NaN
498,NaN,NaN,NaN,NaN,0.274540,NaN


In [210]:
def get_top_n_results(dataframe_results,top_n=1):
    df_top_n = dataframe_results.sort_values(['top_probability','prediction'], ascending=True).groupby('prediction').head(top_n)
    # print(df_top_n)
    accuracy_top_n = df_top_n.right_wrong.sum()/len(df_top_n.right_wrong)
    # evaluation_metrics.get_metrics(df_top_n['prediction_code'],Y_code,metrics=["accuracy"])

    accuracy = { "accuracy in top {}".format(top_n) : round(accuracy_top_n,3)}
    print(accuracy)
    return 

get_top_n_results(df_predictions_probs,top_n = 1)
get_top_n_results(df_predictions_probs,top_n = 2)
get_top_n_results(df_predictions_probs,top_n = 3)
get_top_n_results(df_predictions_probs,top_n = 4)
get_top_n_results(df_predictions_probs,top_n = 5)
get_top_n_results(df_predictions_probs,top_n = 6)
get_top_n_results(df_predictions_probs,top_n = 7)
get_top_n_results(df_predictions_probs,top_n = 8)
get_top_n_results(df_predictions_probs,top_n = 9)
get_top_n_results(df_predictions_probs,top_n = 10)
get_top_n_results(df_predictions_probs,top_n = 11)
get_top_n_results(df_predictions_probs,top_n = 12)
get_top_n_results(df_predictions_probs,top_n = 13)
get_top_n_results(df_predictions_probs,top_n = 14)
get_top_n_results(df_predictions_probs,top_n = 15)
get_top_n_results(df_predictions_probs,top_n = 16)
get_top_n_results(df_predictions_probs,top_n = len(train_dataset))

{'accuracy in top 1': 0.333}
{'accuracy in top 2': 0.333}
{'accuracy in top 3': 0.333}
{'accuracy in top 4': 0.417}
{'accuracy in top 5': 0.333}
{'accuracy in top 6': 0.294}
{'accuracy in top 7': 0.263}
{'accuracy in top 8': 0.286}
{'accuracy in top 9': 0.304}
{'accuracy in top 10': 0.32}
{'accuracy in top 11': 0.296}
{'accuracy in top 12': 0.276}
{'accuracy in top 13': 0.258}
{'accuracy in top 14': 0.303}
{'accuracy in top 15': 0.314}
{'accuracy in top 16': 0.297}
{'accuracy in top 500': 0.274}


# KMeans

In [14]:
km1 = KMeans(n_clusters=len(raw_data[class_col].drop_duplicates()), n_init=25, max_iter = 600, random_state=422)


In [17]:
km1

KMeans(max_iter=600, n_clusters=4, n_init=25, random_state=422)

In [ ]:
km1.fit()